In [15]:
import pandas as pd

In [38]:
df = pd.read_csv("../student_flow_coordinates.csv")

In [39]:
df = df.groupby(df.columns.tolist()).size().reset_index().rename(columns={0:'count'})

In [40]:
df = df.rename(columns={"coordinates_send": "send", "coordinates_host": "receive"})

In [41]:
df["sendLat"] = df.send.apply(lambda x: x.split(",")[0].lstrip("(").strip())
df["sendLon"] = df.send.apply(lambda x: x.split(",")[1].rstrip(")").strip())

In [42]:
df["receiveLat"] = df.receive.apply(lambda x: x.split(",")[0].lstrip("(").strip())
df["receiveLon"] = df.receive.apply(lambda x: x.split(",")[1].rstrip(")").strip())

In [43]:
df = df.drop(columns=["send", "receive"])

In [44]:
ci = pd.read_csv("../geonames.org/countryInfo.txt", delimiter="\t")
ci = ci[["ISO", "ISO-Numeric"]].rename(columns={"Country": "name", "ISO-Numeric": "numeric"})

In [45]:
df = df.merge(ci, left_on="SendingCountry", right_on="ISO", how="left").drop(columns=["ISO"])\
        .rename(columns={"numeric": "sendingNumeric"})
df = df.merge(ci, left_on="ReceivingCountry", right_on="ISO", how="left").drop(columns=["ISO"])\
        .rename(columns={"numeric": "receivingNumeric"})

In [46]:
df = df.rename(columns={"SendingCountry": "sendingCode", "ReceivingCountry": "receivingCode"})

In [47]:
df.sendingCode = df.sendingCode.str.lower()
df.receivingCode = df.receivingCode.str.lower()

**Check the weird ougoing data.**

In [71]:
[lat_a, lon_a] = [72.584117,40.366053]
[lat_b, lon_b] = [25.679281, -36.758695]

de = df[df.sendingCode == "de"].copy()
de["receiveLat"] = de.receiveLat.astype(float)
de["receiveLon"] = de.receiveLon.astype(float)

# Reunion
# French Guiana
# New Caledonia

res = de[~(((lat_b < de.receiveLat) & (de.receiveLat < lat_a)) & ((lon_b < de.receiveLon) & (de.receiveLon < lon_a)))]

In [75]:
res[["Name of Organisation_host", "count"]].to_clipboard(index=False)